# Analyze the BD results

In [1]:
import scvelo as scv
import scanpy
import igraph
import glob, os
import pandas as pd
import numpy as np
import re
from collections import Counter
import anndata

import shutil
    
import h5py
from shutil import copyfile


def copyFiles(files, to):
    for f in files:
        name = os.path.basename( f )
        print( f"copy {f} to {to}" )
        copyfile( f, os.path.join(to, name ) )
    print( "all copied" )

## This now (2023.05.20)

is based on the Rhapsody 1.2.1 (first commit).
The data was created in the TestData folder using the commands

```
../target/release/quantify_rhapsody -r cells.1.Rhapsody_SV_index1_S1_R1_001.fastq.gz -f cells.1.Rhapsody_SV_index1_S1_R2_001.fastq.gz -o BD_results/Rustody_S1 -s mouse  -e 2276_20220531_chang_to_rpl36a_amplicons.fasta -a MyAbSeqP
anel.fasta -m 200 -v v2.96 --gene-kmers 16

../target/release/quantify_rhapsody -r cells.1.Rhapsody_SV_index2_S2_R1_001.fastq.gz -f cells.1.Rhapsody_SV_index2_S2_R2_001.fastq.gz -o BD_results/Rustody_S2 -s mouse  -e 2276_20220531_chang_to_rpl36a_amplicons.fasta -a MyAbSeqPanel.fasta -m 200 -v v2.96 --gene-kmers 16
```

The --gene_kmers makes a huge difference - still.
This is the setting where I hat more antibody tags out.

I am shure this mapper is more stringent than my first implementation as this here actually checks two consecutive DNA fragments 8bp +16bp. Hence the matching are is bigger than in version 0.x. 

## Overall result

Looks acceptable. I identify less reads as containing sequence, but the result still looks more or less the same.
I need to look into that in more detail.

In [2]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

scvelo==0.2.4
scanpy==1.8.2
igraph==0.9.9
pandas==1.5.0.dev0+268.gbe8d1ec88
numpy==1.21.6
re==2.2.1
anndata==0.8.0
h5py==3.7.0


In [3]:
files = [ 'Combined_S1Subset_DBEC_MolsPerCell.csv', 'Combined_S2Subset_DBEC_MolsPerCell.csv' ]
sinfoF = [ 'S1Subset_Sample_Tag_Calls.csv', 'S2Subset_Sample_Tag_Calls.csv']
sname = ['Index1', 'Index2' ]

ofiile = "BD_analyzed.h5ad"

In [4]:
df = pd.read_csv(files[0], skiprows=7, index_col=0)
df.columns[0:20]

Index(['CD117:2B8|Kit|AMM2023|pAbO', 'CD19|Cd19|AMM2007|pAbO',
       'CD25:PC61|Il2ra|AMM2012|pAbO', 'CD5|Cd5|AMM2043|pAbO',
       'IgM|Ighm|AMM2031|pAbO', '2810417H13Rik', 'Ada', 'Adgre1', 'Adgrg3',
       'Aicda', 'Alas2', 'Anxa5', 'Apoe', 'Aqp9', 'Arg1', 'Arg2', 'Arid3a',
       'Arl4c', 'Atf6b', 'Atg5'],
      dtype='object')

In [5]:
m = np.array([ s.__contains__ ("|") for s in df.columns ]).sum()

In [6]:
adata = anndata.AnnData(X = df.iloc[0:,m:])

In [7]:
adata.var_names
adata.obs['sample'] = sname[0]

In [8]:
df = pd.read_csv(files[1], skiprows=7, index_col=0)
tmp = anndata.AnnData(X = df.iloc[0:,m:])
tmp.obs['sample'] = sname[1]
adata = adata.concatenate( tmp )
BD_Data = adata
del adata

In [9]:
os.name

'posix'

In [10]:
! source ~/.cargo/env &&  cargo build -r  

  --> src/bin/quantify_rhapsody_multi.rs:82:4
   |
82 | fn check_file_existence(file_path: &str, option: &str, errors: &mut Vec<String>) {
   |    ^^^^^^^^^^^^^^^^^^^^
   |
   = note: `#[warn(dead_code)]` on by default

    Finished release [optimized] target(s) in 0.04s


In [11]:
if os.path.exists("Rustody_S1"):
    shutil.rmtree('Rustody_S1')
if not os.path.exists("Rustody_S1"):
    f1 =  os.path.join( "..", "cells.1.Rhapsody_SV_index1_S1_R1_001.fastq.gz")
    f2 =  os.path.join( "..", "cells.1.Rhapsody_SV_index1_S1_R2_001.fastq.gz")
    ab =  os.path.join( "..", "MyAbSeqPanel.fasta")
    ex =  os.path.join( "..", "2276_20220531_chang_to_rpl36a_amplicons.fasta")
    exe = os.path.join( "..", '..', 'target', 'release', 'quantify_rhapsody_multi' )
    if os.name == 'nt': # windows...
        exe = exe + '.exe'
    print ( f"{exe} -r {f1} -f {f2} -o Rustody_S1 -s mouse  -e {ex} -a {ab} -m 200 -v v2.96")
    ! {exe} -r {f1} -f {f2} -o Rustody_S1 -s mouse  -e {ex} -a {ab} -m 200 -v v2.96

../../target/release/quantify_rhapsody_multi -r ../cells.1.Rhapsody_SV_index1_S1_R1_001.fastq.gz -f ../cells.1.Rhapsody_SV_index1_S1_R2_001.fastq.gz -o Rustody_S1 -s mouse  -e ../2276_20220531_chang_to_rpl36a_amplicons.fasta -a ../MyAbSeqPanel.fasta -m 200 -v v2.96
New output directory created successfully!
Analysis will stop after having processed 18446744073709551615 fastq entries containing a cell info

init models
the log file: Mapping_log.txt
Changing the expression start gene id to 462
After indexing all fastq files we have the following indices:
the mRNA index:
I have 41039 kmers for 462 genes with 0.16375615% duplicate entries
gene names like 'Rpl36a'
gene_ids range from Some(0) to Some(461)

the sample id index:
I have 334 kmers for 12 genes with 0.09677419% duplicate entries
gene names like 'Sample12'
gene_ids range from Some(0) to Some(11)

and the antibodies index:
I have 143 kmers for 5 genes with 0% duplicate entries
gene names like 'IgM'
gene_ids range from Some(0) to So

In [12]:
exe

'../../target/release/quantify_rhapsody_multi'

In [13]:
if os.path.exists("Rustody_S2"):
    shutil.rmtree('Rustody_S2')
if not os.path.exists("Rustody_S2"):
    f1 =  os.path.join( "..", "cells.1.Rhapsody_SV_index2_S2_R1_001.fastq.gz")
    f2 =  os.path.join( "..", "cells.1.Rhapsody_SV_index2_S2_R2_001.fastq.gz")
    ab =  os.path.join( "..", "MyAbSeqPanel.fasta")
    ex =  os.path.join( "..", "2276_20220531_chang_to_rpl36a_amplicons.fasta")
    print ( f"{exe} -r {f1} -f {f2} -o Rustody_S2 -s mouse  -e {ex} -a {ab} -m 200 -v v2.96")
    ! {exe} -r {f1} -f {f2} -o Rustody_S2 -s mouse  -e {ex} -a {ab} -m 200 -v v2.96

../../target/release/quantify_rhapsody_multi -r ../cells.1.Rhapsody_SV_index2_S2_R1_001.fastq.gz -f ../cells.1.Rhapsody_SV_index2_S2_R2_001.fastq.gz -o Rustody_S2 -s mouse  -e ../2276_20220531_chang_to_rpl36a_amplicons.fasta -a ../MyAbSeqPanel.fasta -m 200 -v v2.96
New output directory created successfully!
Analysis will stop after having processed 18446744073709551615 fastq entries containing a cell info

init models
the log file: Mapping_log.txt
Changing the expression start gene id to 462
After indexing all fastq files we have the following indices:
the mRNA index:
I have 41039 kmers for 462 genes with 0.16375615% duplicate entries
gene names like 'Rpl36a'
gene_ids range from Some(0) to Some(461)

the sample id index:
I have 334 kmers for 12 genes with 0.09677419% duplicate entries
gene names like 'Sample12'
gene_ids range from Some(0) to Some(11)

and the antibodies index:
I have 143 kmers for 5 genes with 0% duplicate entries
gene names like 'IgM'
gene_ids range from Some(0) to So

In [16]:
def readRustodyExpression(path, name):
    print(f"reading Rustody expression from path {path}/BD_Rhapsody_expression/")
    this = scanpy.read_10x_mtx( path+'/BD_Rhapsody_expression/' )
    this.obs['sample'] = name
    obs1 = pd.read_csv( path+'/SampleCounts.tsv', sep="\t")
    this.obs = this.obs.merge( obs1, left_index= True, right_on = 'CellID' )
    this.obs_names = this.obs['CellID'] + "_" +  this.obs['sample']
    this = this[this.obs['AssignedSampleName'] != "na"]
    this.obs['AssignedSampleName'] = this.obs['AssignedSampleName'] + "_" + this.obs['sample']
    return(this)

In [17]:
Rustody = readRustodyExpression( 'Rustody_S1', 'Index1')
Rustody = Rustody.concatenate(readRustodyExpression( 'Rustody_S2', 'Index2'))

reading Rustody expression from path Rustody_S1/BD_Rhapsody_expression/
reading Rustody expression from path Rustody_S2/BD_Rhapsody_expression/


In [18]:
Rustody.obs['cellID'] =  [ re.findall(r'\d+', n)[0] for n in Rustody.obs_names]
BD_Data.obs['cellID'] =  [ re.findall(r'\d+', n)[0] for n in BD_Data.obs_names]

In [19]:
Rustody.obs['cellname'] = [Rustody.obs['cellID'][id] + "_" + Rustody.obs['sample'][id] for id in range( len(Rustody.obs_names))]
BD_Data.obs['cellname'] = [BD_Data.obs['cellID'][id] + "_" + BD_Data.obs['sample'][id] for id in range( len(BD_Data.obs_names))]
Rustody.obs['source'] = "Rustody"
BD_Data.obs['source'] = "BD"

In [20]:
BD_Data.obs['cellname']

Cell_Index
7243072-0      7243072_Index1
4753157-0      4753157_Index1
8267932-0      8267932_Index1
7405530-0      7405530_Index1
7983433-0      7983433_Index1
                   ...       
1509915-1      1509915_Index2
10329694-1    10329694_Index2
11079622-1    11079622_Index2
2522963-1      2522963_Index2
1799438-1      1799438_Index2
Name: cellname, Length: 130, dtype: object

In [21]:
'4753157_Index1' in BD_Data.obs['cellname']

False

In [22]:
Rustody.obs['cellname']

Cell1164_Index1-0            1164_Index1
Cell30353_Index1-0          30353_Index1
Cell305363_Index1-0        305363_Index1
Cell314929_Index1-0        314929_Index1
Cell1509915_Index1-0      1509915_Index1
                              ...       
Cell7972283_Index2-1      7972283_Index2
Cell8116618_Index2-1      8116618_Index2
Cell8143512_Index2-1      8143512_Index2
Cell9732142_Index2-1      9732142_Index2
Cell10329694_Index2-1    10329694_Index2
Name: cellname, Length: 76, dtype: object

In [23]:
 [name  in BD_Data.obs['cellname'].values for name in Rustody.obs['cellname'].values ]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [24]:
Rustody_4test = Rustody[  [name  in BD_Data.obs['cellname'].values for name in Rustody.obs['cellname'].values ] ]
Rustody_4test

View of AnnData object with n_obs × n_vars = 75 × 244
    obs: 'sample', 'CellID', 'Sample1', 'Sample10', 'Sample11', 'Sample12', 'Sample2', 'Sample3', 'Sample4', 'Sample5', 'Sample6', 'Sample7', 'Sample8', 'Sample9', 'AssignedSampleName', 'FractionTotal', 'n', 'dist to nr.2 [%max]', 'batch', 'cellID', 'cellname', 'source'
    var: 'gene_ids', 'feature_types'

In [25]:
BD_Data_4test = BD_Data[  [name  in Rustody.obs['cellname'].values for name in BD_Data.obs['cellname'].values ] ]
BD_Data_4test

View of AnnData object with n_obs × n_vars = 75 × 462
    obs: 'sample', 'batch', 'cellID', 'cellname', 'source'

In [33]:
Rustody_4test.var_names

Index(['Ccnd2', 'Ctsd', 'Il15ra', 'Itgam', 'Cdkn1a', 'Mcl1', 'Ctla4', 'Anxa5',
       'Ccr7', 'Il18',
       ...
       'Ybx3', 'Thbd', 'Cd28', 'Tigit', 'Fyn', 'Dock8', 'Cd22', 'Fbl', 'Ebf1',
       'H2-DMa'],
      dtype='object', length=244)

In [41]:
gene =  'Il18'
pearsonr( np.array(BD_Data_4test[:,gene].X).flatten(), 
               np.array( Rustody_4test[:,gene].X.todense()).flatten() ) 


(-0.04845015831115095, 0.6797635360781131)

In [50]:
BD_Data_4test.obs_names.sort_values()

Index(['10175248-0', '10329694-1', '10330078-0', '10481686-0', '1164-0',
       '14029829-0', '1509915-0', '1509915-1', '157841-1', '1787564-1',
       '1797122-0', '1799438-0', '1799438-1', '1949217-0', '2079807-0',
       '2090208-0', '2099766-0', '25040-1', '2510987-0', '2522515-0',
       '2522963-1', '2538660-0', '2543289-0', '30353-0', '305363-0',
       '3103568-1', '314929-0', '3569368-1', '4283522-1', '4305474-1',
       '4753157-0', '4885340-1', '5018926-1', '5160998-0', '5166047-1',
       '5328779-0', '5331545-1', '5343016-0', '5463585-0', '5463585-1',
       '5753122-1', '5784982-0', '5926310-0', '6069190-0', '6193184-1',
       '6215132-0', '6228103-1', '6348767-1', '6367528-0', '6658620-0',
       '6785717-0', '6800349-0', '7107103-1', '7236933-1', '7243072-0',
       '7243433-1', '7405530-0', '7546029-1', '7693075-0', '7695391-0',
       '7972283-1', '7983433-0', '8116618-0', '8116618-1', '8143512-1',
       '8267932-0', '8405806-0', '8416901-0', '8713787-0', '9152684-0

In [52]:
BD_Data_4test = BD_Data_4test[ BD_Data_4test.obs_names.sort_values() ]
BD_Data_4test.obs_names

Index(['10175248-0', '10329694-1', '10330078-0', '10481686-0', '1164-0',
       '14029829-0', '1509915-0', '1509915-1', '157841-1', '1787564-1',
       '1797122-0', '1799438-0', '1799438-1', '1949217-0', '2079807-0',
       '2090208-0', '2099766-0', '25040-1', '2510987-0', '2522515-0',
       '2522963-1', '2538660-0', '2543289-0', '30353-0', '305363-0',
       '3103568-1', '314929-0', '3569368-1', '4283522-1', '4305474-1',
       '4753157-0', '4885340-1', '5018926-1', '5160998-0', '5166047-1',
       '5328779-0', '5331545-1', '5343016-0', '5463585-0', '5463585-1',
       '5753122-1', '5784982-0', '5926310-0', '6069190-0', '6193184-1',
       '6215132-0', '6228103-1', '6348767-1', '6367528-0', '6658620-0',
       '6785717-0', '6800349-0', '7107103-1', '7236933-1', '7243072-0',
       '7243433-1', '7405530-0', '7546029-1', '7693075-0', '7695391-0',
       '7972283-1', '7983433-0', '8116618-0', '8116618-1', '8143512-1',
       '8267932-0', '8405806-0', '8416901-0', '8713787-0', '9152684-0

In [53]:
Rustody_4test = Rustody_4test[Rustody_4test.obs_names.sort_values()]

In [54]:
np.array(BD_Data_4test[:,gene].X).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [55]:
np.array(Rustody_4test[:,gene].X.todense()).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [56]:
from scipy.stats.stats import pearsonr   
cmp = pd.DataFrame(
    [ pearsonr( np.array(BD_Data_4test[:,gene].X).flatten(), 
               np.array( Rustody_4test[:,gene].X.todense()).flatten() ) for gene in Rustody_4test.var_names if gene in BD_Data_4test.var_names ]  )
cmp['gname'] = [gene  for gene in Rustody_4test.var_names if gene in BD_Data_4test.var_names ]
cmp

,0,1,gname
0,1.000000,0.000000e+00,Ccnd2
1,0.973065,2.879407e-48,Ctsd
2,0.929403,2.502492e-33,Il15ra
3,1.000000,0.000000e+00,Itgam
4,1.000000,0.000000e+00,Cdkn1a
...,...,...,...
239,1.000000,0.000000e+00,Dock8
240,0.985371,7.531320e-58,Cd22
241,0.983262,9.896981e-56,Fbl
242,0.739830,3.391389e-14,Ebf1


In [62]:
vals = np.array(BD_Data_4test[:,'Ebf1'].X).flatten() == 2
[ id for id in range( len(vals) ) if vals[id] ]

[59]

In [76]:
sorted_var_names = sorted(Rustody_4test.var_names)
BD_Data_4test = BD_Data_4test[:, sorted_var_names]
Rustody_4test = Rustody_4test[:, sorted_var_names]

In [77]:
pearsonr( np.array(Rustody_4test [:, 59].X.toarray().flatten()),
       np.array(BD_Data_4test [:, 59].X.toarray().flatten()))

(0.9604759666203988, 2.7698182615922647e-42)

In [78]:
np.array(Rustody_4test [:, 59].X.toarray().flatten())

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 2., 0., 1., 0., 2., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 1., 2., 0., 1., 0., 0., 2., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 2., 0., 2., 0., 1., 0., 1., 1., 0., 2., 0., 1., 0., 0.,
       0., 0., 0., 0., 2., 0., 0.], dtype=float32)

In [79]:
np.array(BD_Data_4test [:, 59].X.toarray().flatten())

array([0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 2., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 2., 0., 1., 0., 3., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 1., 2., 0., 1., 0., 0., 2., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 2., 0., 1., 0., 1., 1., 0., 2., 0., 1., 0., 0.,
       0., 0., 0., 0., 2., 0., 0.], dtype=float32)

In [81]:
from scipy.stats.stats import pearsonr   
cmp = pd.DataFrame(
    [ pearsonr( np.array(BD_Data_4test[:,gene].X).flatten(), 
               np.array( Rustody_4test[:,gene].X.todense()).flatten() ) for gene in Rustody_4test.var_names if gene in BD_Data_4test.var_names ]  )
cmp['gname'] = [gene  for gene in Rustody_4test.var_names if gene in BD_Data_4test.var_names ]
cmp

,0,1,gname
0,0.994028,5.487680e-72,2810417H13Rik
1,0.990703,5.384115e-65,Ada
2,0.955216,2.413487e-40,Anxa5
3,0.997755,1.805195e-87,Apoe
4,0.980786,1.458155e-53,Arid3a
...,...,...,...
239,0.996302,1.447880e-79,Vpreb3
240,0.979166,2.720688e-52,Vps28
241,0.998216,4.153936e-91,Xbp1
242,0.979472,1.594057e-52,Ybx3


In [86]:
not_good = [id for id in range( len( cmp[0])) if cmp[0][id] < 0.9]
cmp.iloc[not_good, :]

,0,1,gname
14,0.336416,3.167011e-03,Bcl2l11
41,0.216720,6.181983e-02,Cd36
42,0.127551,2.754801e-01,Cd37
50,0.008318,9.435359e-01,Cd74
60,0.866443,1.018606e-23,Cnot2
65,0.218218,5.999690e-02,Cxcr4
70,0.755025,5.079868e-15,Dpp4
73,0.739830,3.391389e-14,Ebf1
83,0.696733,3.866365e-12,Fbxl12
125,0.859990,5.060482e-23,Il12a


In [88]:
gene  = 'Bcl2l11'
print("BD")
print (np.array(BD_Data_4test[:,gene].X).flatten() )
print("Rustody")
print(np.array(Rustody_4test[:,gene].X.todense()).flatten())

BD
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
Rustody
[0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 1. 0. 2. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0.]


In [31]:
Counter( Rustody_4test.obs['sample'])

Counter({'Index1': 47, 'Index2': 28})

In [80]:
import plotly.express as px
df = pd.DataFrame( 
    { 
        'BD_I1' :adata[adata.obs['sample'] == 'S1_BD'].obs['log1p_total_counts'],
        'BD_I2' :adata[adata.obs['sample'] == 'S2_BD'].obs['log1p_total_counts'],
        'Rustody_I1' :adata[adata.obs['sample'] == 'S1_Rustody'].obs['log1p_total_counts'],
        'Rustody_I2' :adata[adata.obs['sample'] == 'S2_Rustody'].obs['log1p_total_counts']
    })
fig = px.violin(df, y= df.columns )
fig.show() 


NameError: name 'adata' is not defined

In [ ]:
adata[:,"Cnot2"][adata.obs['sample'] == 'S1_Rustody'].X.todense().sum()

In [ ]:
scanpy.pp.filter_genes(adata, min_counts=1 )
scanpy.pp.filter_cells(adata, min_genes=20 )
Counter( adata.obs['sample'])

In [ ]:
adata.obs['cellID'] =  [ re.findall(r'\d+', n)[0] for n in adata.obs_names] 

In [ ]:
adata.obs['cellID']  = adata.obs['cellID'].astype('str')

In [ ]:
Counter([ name in adata[adata.obs['sample'] == 'S1_BD'].obs['cellID'].values for name in adata[adata.obs['sample'] == 'S1_Rustody'].obs['cellID'] ])

In [ ]:
Counter([ name in adata[adata.obs['sample'] == 'S1_BD'].obs['cellID'].values 
         for name in adata[adata.obs['sample'] == 'S1_Rustody'].obs['cellID'] ])


In [ ]:
Counter([ name in adata[adata.obs['sample'] == 'S1_Rustody'].obs['cellID'].values 
         for name in adata[adata.obs['sample'] == 'S1_BD'].obs['cellID'] ])


In [ ]:
[[ name, name in adata[adata.obs['sample'] == 'S1_Rustody'].obs['cellID'].values ]
         for name in adata[adata.obs['sample'] == 'S1_BD'].obs['cellID'] ]

In [ ]:
! bd_cell_id_2_seq --id 12842538 --version v2.96

In [ ]:
adata[adata.obs['sample'] == 'S1_Rustody'].obs['cellID'] 

In [ ]:
.astype('str').astype('str').astype('str')

In [ ]:
"1164" in adata[adata.obs['sample'] == 'S1_BD'].obs['cellID'].values 

In [ ]:
dat = adata[ adata.obs['cellID'] == '4753157' ].X.todense()
ids = [id  for id in range(dat.shape[1]) if not dat[0,id] == dat[1,id] ]
bools = [ not dat[0,id] == dat[1,id]  for id in range(dat.shape[1]) ]
adata.var.loc[bools,:]

In [ ]:
adata

In [ ]:
diff_val_BD = [ int(n) for n in dat[0,bools].transpose()]

In [ ]:
diff_val_Rhapsody = [ int(n) for n in dat[1,bools].transpose()]

In [ ]:
import plotly.express as px
df = pd.DataFrame( { 'BD' :np.log1p(diff_val_BD), 'Rhapsody' : np.log1p(diff_val_Rhapsody)})
fig = px.violin(df, y= df.columns )
fig.show()

In [ ]:
adata[ adata.obs['cellID'] == '4753157' ].obs

In [ ]:
dat.shape[1]

In [ ]:
scanpy.pp.calculate_qc_metrics( adata, inplace=True, log1p=True, percent_top= [10])

In [ ]:
scanpy.pp.filter_cells(adata, min_counts=400 )
scanpy.pp.downsample_counts(adata, counts_per_cell= 400 )
scv.pp.log1p(adata)
Counter( adata.obs['sample'])

In [ ]:
scanpy.pp.neighbors(adata)
dimensions = 2
scanpy.tl.umap(adata,n_components= dimensions)

In [ ]:
scv.pl.scatter(adata, color='sample', figsize =(7,5), legend_loc='right margin')

In [ ]:
scanpy.tl.louvain(adata)

In [ ]:
scv.pl.scatter(adata, color='louvain', figsize =(7,5), legend_loc='right margin')

In [ ]:
adata

In [ ]:
adata.obs.pivot_table(values = "n_counts", index = "louvain", columns="sample", aggfunc='count')

In [ ]:
adata.obs['n_genes' ] = adata.obs['n_genes' ].astype('int32')

In [ ]:
adata

In [ ]:
scanpy.pl.violin(adata, [ 'n_genes', 'log1p_total_counts' ], 'sample')

In [ ]:
key_added = "louvain"
scanpy.tl.rank_genes_groups(
    adata, 
    groupby   = 'louvain',
    key_added = key_added,
    method    = 'wilcoxon',
)

scanpy.pl.rank_genes_groups(adata, key = key_added )

In [ ]:
scv.pl.scatter(adata, color=['Igkc', 'Igha', 'Ighm'], figsize =(7,5), legend_loc='right margin')

In [ ]:
scv.pl.scatter(adata, color=[ 'Ighg1', 'Ighg2b', 'Ighg3' ], figsize =(7,5), legend_loc='right margin')

In [ ]:
names = np.array(adata.obs['sample'].unique()).astype('str')
names.sort()
names

In [ ]:
for n in names:
    print (n)
    scv.pl.scatter(adata[adata.obs['sample'] == n], color=[ 'Ighg1', 'Ighg2b', 'Ighg3' ], figsize =(7,5), legend_loc='right margin')

In [ ]:
for n in names:
    print (n)
    scv.pl.scatter(adata[adata.obs['sample'] == n], color=[ 'Top2a', 'Tmem97', 'Jchain' ], figsize =(7,5), legend_loc='right margin')